In [1]:
import os
import pandas as pd

import numpy as np
import scanpy as sc
import mudata as md
import anndata as ad

import warnings
warnings.filterwarnings('ignore')
from scipy import sparse

In [2]:
mdata = md.read_h5mu('../../results/preprocessing_data/multiome_train.h5mu')

In [3]:
mdata

MuData object with n_obs × n_vars = 25551 × 158205
  var:	'gene_id', 'feature_type', 'genome', 'interval'
  2 modalities
    rna:	25551 x 22787
      obs:	'cell_type', 'donor_id'
      var:	'gene_id', 'feature_type', 'genome', 'interval'
      layers:	'counts', 'normalized_count'
    atac:	25551 x 135418
      obs:	'cell_type', 'donor_id'
      var:	'gene_id', 'feature_type', 'genome', 'interval'
      layers:	'counts', 'normalized_count'

In [4]:
adata = mdata['rna']

In [5]:
adata.obs

,cell_type,donor_id
obs_id,,
000225c1151ab841,B cells,donor_0
0003c40a54367871,T cells CD4+,donor_2
0004bf574b822c3c,T cells CD4+,donor_2
000d59b5478f28e2,B cells,donor_0
0011b7473923d7b5,NK cells,donor_2
...,...,...
fff2ca1f64c10339,T cells CD4+,donor_0
fff87e64f509b570,T cells CD4+,donor_0
fff9778b31bc2539,Myeloid cells,donor_2


In [6]:
def sum_by(adata: ad.AnnData, col: str) -> ad.AnnData:
    """
    Adapted from this forum post: 
    https://discourse.scverse.org/t/group-sum-rows-based-on-jobs-feature/371/4
    """
    
    assert pd.api.types.is_categorical_dtype(adata.obs[col])

    # sum `.X` entries for each unique value in `col`
    cat = adata.obs[col].values
    indicator = sparse.coo_matrix(
        (
            np.broadcast_to(True, adata.n_obs),
            (cat.codes, np.arange(adata.n_obs))
        ),
        shape=(len(cat.categories), adata.n_obs),
    )
    sum_adata = ad.AnnData(
        indicator @ adata.X,
        var=adata.var,
        obs=pd.DataFrame(index=cat.categories),
        dtype=adata.X.dtype,
    )
    
    # copy over `.obs` values that have a one-to-one-mapping with `.obs[col]`
    obs_cols = adata.obs.columns
    obs_cols = list(set(adata.obs.columns) - set([col]))
    
    one_to_one_mapped_obs_cols = []
    nunique_in_col = adata.obs[col].nunique()
    for other_col in obs_cols:
        if len(adata.obs[[col, other_col]].drop_duplicates()) == nunique_in_col:
            one_to_one_mapped_obs_cols.append(other_col)

    joining_df = adata.obs[[col] + one_to_one_mapped_obs_cols].drop_duplicates().set_index(col)
    assert (sum_adata.obs.index == sum_adata.obs.join(joining_df).index).all()
    sum_adata.obs = sum_adata.obs.join(joining_df)
    sum_adata.obs.index.name = col
    sum_adata.obs = sum_adata.obs.reset_index()
    sum_adata.obs.index = sum_adata.obs.index.astype('str')

    return sum_adata

In [7]:
adata.obs['donor_id_cell_type'] = adata.obs['donor_id'].astype('str') + '_' + adata.obs['cell_type'].astype('str')
adata.obs['donor_id_cell_type'] = adata.obs['donor_id_cell_type'].astype('category')

bulk_adata = sum_by(adata, 'donor_id_cell_type')
bulk_adata.obs = bulk_adata.obs.drop(columns=['donor_id_cell_type'])
bulk_adata.X = np.array(bulk_adata.X.todense())
bulk_adata.X = bulk_adata.X.astype('float64')
bulk_adata = bulk_adata.copy()

In [18]:
sc.pp.normalize_total(bulk_adata)
sc.pp.log1p(bulk_adata)

In [19]:
bulk_adata

AnnData object with n_obs × n_vars = 18 × 22787
    obs: 'cell_type', 'donor_id'
    var: 'gene_id', 'feature_type', 'genome', 'interval'
    uns: 'log1p'

In [20]:
bulk_adata.X

array([[4.37172094, 4.9164097 , 3.2178806 , ..., 5.30917139, 4.56286929,
        5.83745702],
       [4.45324852, 4.1591306 , 5.53101532, ..., 5.39315989, 6.25456548,
        6.53215626],
       [4.42667607, 4.8281485 , 6.72776423, ..., 4.90757553, 5.49631904,
        6.23913446],
       ...,
       [4.36812201, 4.2575074 , 5.81590758, ..., 5.53796327, 5.2827902 ,
        6.19482625],
       [4.31898228, 3.92015183, 4.7560489 , ..., 4.88850477, 5.33207643,
        6.06193119],
       [4.6074159 , 4.6074159 , 4.10321997, ..., 5.47704739, 4.867475  ,
        6.26699477]])

In [23]:
co_exp = np.corrcoef(bulk_adata.X.transpose())

In [24]:
co_exp.shape

(22787, 22787)

In [26]:
co_exp[:5, :5]

array([[ 1.        ,  0.3845928 , -0.27820491, -0.35130291, -0.33729904],
       [ 0.3845928 ,  1.        , -0.05219028, -0.17901279, -0.24170707],
       [-0.27820491, -0.05219028,  1.        ,  0.6811595 ,  0.36590315],
       [-0.35130291, -0.17901279,  0.6811595 ,  1.        , -0.18578469],
       [-0.33729904, -0.24170707,  0.36590315, -0.18578469,  1.        ]])

In [27]:
# keep important edges
co_exp[abs(co_exp) < 0.5] = 0

In [28]:
co_exp[:5, :5]

array([[1.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 1.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 1.       , 0.6811595, 0.       ],
       [0.       , 0.       , 0.6811595, 1.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 1.       ]])

In [30]:
co_exp[:30, :30]

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.53041956,  0.        ,
         0.        ,  0.        ,  0.        ,  0.51385217,  0.        ,
         0.        ,  0.        , -0.67180718,  0.        ,  0.        ,
         0.64900548,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.63614731,
         0.        ,  0.        ,  0.        ,  0.50707365,  0.        ,
         0.        ,  0.        , -0.51875425,  0.51936079,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.6811595 ,  0.        ,
         0.        ,  0.61702018,  0.        ,  0